In [1]:
import pandas as pd
import numpy as np
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

input_path = r"C:\Users\Klucly\Desktop\competition\train.csv"
output_path = "../data/preprocessed_data.csv"


In [2]:
df = pd.read_csv(input_path)

C:\Users\Klucly\AppData\Local\Temp\ipykernel_33100\1758342947.py:1: DtypeWarning: Columns (24,25,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_path)


In [3]:
df.isnull().sum()

transaction_id                         0
is_fraud                               0
created_at                             0
is_subscription                        0
transaction_type                       0
currency_amount                        0
currency_id                            0
amount_scaled                          0
merchant_customer_id             5283914
merchant_customer_email               10
merchant_customer_phone         35150355
merchant_customer_first_name    31914541
merchant_customer_last_name     33244135
merchant_country                       2
merchant_city                   35358214
merchant_language                8420113
ip_address                             2
platform                               0
merchant_id                            0
merchant_shop_id                       0
merchant_shop_name                     0
is_secured                             0
order_number                    34814741
ip_country                        139510
is_verified     

In [4]:
missing_features_percentage = (df.isnull().sum()/len(df))*100
missing_features_percentage
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36387900 entries, 0 to 36387899
Data columns (total 43 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   transaction_id                uint64 
 1   is_fraud                      int64  
 2   created_at                    object 
 3   is_subscription               bool   
 4   transaction_type              object 
 5   currency_amount               float64
 6   currency_id                   int64  
 7   amount_scaled                 int64  
 8   merchant_customer_id          object 
 9   merchant_customer_email       object 
 10  merchant_customer_phone       object 
 11  merchant_customer_first_name  object 
 12  merchant_customer_last_name   object 
 13  merchant_country              object 
 14  merchant_city                 object 
 15  merchant_language             object 
 16  ip_address                    object 
 17  platform                      object 
 18  merchant_id         

In [5]:
df = df.dropna(axis=1, thresh=0.22*len(df))

In [6]:
is_fraud = df.pop("is_fraud")
transaction_id = df.pop("transaction_id")

In [7]:
# is_fraud.info()

In [8]:
df = df.drop(columns=["user_agent", "traffic_source", "card_holder_first_name", "card_holder_last_name"])

In [9]:
categorical_features = df.select_dtypes(include=["object"]).columns
df[categorical_features] = df[categorical_features].fillna("Missing")

numerical_features = df.select_dtypes(include=["int64", "uint64", "float64"]).columns
df[numerical_features] = df[numerical_features].fillna(0)
# df = df.dropna(subset=numerical_features)


## Handle outliers

In [10]:
df.describe()

,currency_amount,currency_id,amount_scaled,merchant_id,merchant_shop_id,card_exp_relative
count,3.638790e+07,3.638790e+07,3.638790e+07,3.638790e+07,3.638790e+07,3.638790e+07
mean,1.749441e+05,1.224711e+02,8.425774e+04,9.932898e+18,1.419520e+04,4.291868e+01
std,1.772893e+07,3.093257e+01,1.310226e+07,4.753952e+18,4.525534e+03,2.403203e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,8.627844e+16,9.000000e+00,-6.600000e+01
25%,2.022300e+03,1.220000e+02,6.990000e+02,5.225729e+18,1.456300e+04,2.600000e+01
50%,4.183650e+03,1.220000e+02,1.999000e+03,8.541995e+18,1.511500e+04,4.300000e+01
75%,8.098650e+03,1.410000e+02,3.895000e+03,1.457549e+19,1.676000e+04,5.900000e+01
max,2.899103e+09,1.680000e+02,2.147484e+09,1.840464e+19,1.932000e+04,2.404500e+04


In [11]:
num_cols = df.select_dtypes(include=["number"]).columns
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.clip(df[col], lower_bound, upper_bound)

In [12]:
df.describe()

,currency_amount,currency_id,amount_scaled,merchant_id,merchant_shop_id,card_exp_relative
count,3.638790e+07,3.638790e+07,3.638790e+07,3.638790e+07,3.638790e+07,3.638790e+07
mean,6.038300e+03,1.271731e+02,2.586096e+03,9.932898e+18,1.513851e+04,4.277804e+01
std,5.608110e+03,1.946215e+01,2.254660e+03,4.753952e+18,1.870391e+03,2.256944e+01
min,0.000000e+00,9.350000e+01,0.000000e+00,8.627844e+16,1.126750e+04,-2.350000e+01
25%,2.022300e+03,1.220000e+02,6.990000e+02,5.225729e+18,1.456300e+04,2.600000e+01
50%,4.183650e+03,1.220000e+02,1.999000e+03,8.541995e+18,1.511500e+04,4.300000e+01
75%,8.098650e+03,1.410000e+02,3.895000e+03,1.457549e+19,1.676000e+04,5.900000e+01
max,1.721318e+04,1.680000e+02,8.689000e+03,1.840464e+19,1.932000e+04,1.085000e+02


## Replace rare categories

In [13]:
unbalanced_categories = ['merchant_country', 'merchant_language', 'ip_country', 'platform', 'cardbrand', 'cardcountry']
length = len(df)
for categ in unbalanced_categories:
    if categ in df.columns:
        category_counts = df[categ].value_counts()
        threshold = 0.0001 * length
        df[categ] = df[categ].apply(lambda value: value if category_counts.get(value, 0) > threshold else 'other')

## Process datetime features 

In [14]:
if 'created_at' in df.columns:
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
    df['month_creating'] = df['created_at'].dt.month
    df['week_day_creating'] = df['created_at'].dt.dayofweek
    df.drop('created_at', axis=1, inplace=True)

## Numerical features scaling

In [15]:
scaler = MinMaxScaler()
num_features = df.select_dtypes(include=["number"]).columns.tolist()
scaler.fit(df[num_features])

MinMaxScaler()

In [16]:
df[num_features] = scaler.transform(df[num_features])

In [17]:
df.head()

,is_subscription,transaction_type,currency_amount,currency_id,amount_scaled,merchant_customer_id,merchant_customer_email,merchant_country,merchant_language,ip_address,...,payment_type,card_id,bank,cardbrand,cardcountry,cardtype,bin,card_exp_relative,month_creating,week_day_creating
0,False,first,0.078350,0.0,0.000921,Missing,db30f9b09fc6db70bd69492bad0993610e034dd0aaf196...,USA,Missing,05ebef83d319bc0e9f8f3e5792ed1e7f9b5e93ac61f356...,...,Missing,55b9df24b0ee3d556bd9d92e3e96cd9cd2020aa889a099...,Intl Hdqtrs-Center Owned,VISA,IND,DEBIT,82178912e9cb7cdfe4397e288172d2ddb5b4ca24de4ad7...,0.625000,0.0,0.000000
1,True,first,0.011686,0.0,0.011279,6b5907bb6b2a4db1e1d8e29ecda26c958148c869ae21e8...,4429eb1d2902632d67042cd8b04855c12499684b25fe40...,AUS,en,efdd674a78604c0efb156dbad042dbe633c08338a1a988...,...,Missing,37ebdd04d53f8bbda6cdc8a67a63a3f07178697cc4b81b...,National Australia Bank Limited,VISA,AUS,CREDIT,c84a4f90930cf35cb2718e49919df4f2400347c3e28e5c...,0.526515,0.0,1.000000
2,True,first,0.011686,0.0,0.011394,6f512fcdad63bdcac2fb2283d54e19cfb37b3714d38786...,af46d0225af51b3083c33d1f31e380e07cfa6d7256f30d...,AUS,en,fdb9dd738153f7893179977c6814f916260ef5e4d8a424...,...,Missing,2a7e39132540646a182b136943d02522d6990e9fdedf09...,National Australia Bank Limited,VISA,AUS,DEBIT,a321caacb48f0dd4ac473a76fb7a8eaa54f0c88d409594...,0.488636,0.0,0.333333
3,True,first,0.062664,0.0,0.060536,2ae2b02e168b5614f46a2b94715562012cf01f405b4da2...,53514d7fdcf50004badc5914c5faaad5311e94b9596c90...,AUS,en,1b487bab850c13007c4b0cecfa3003285e53e676bddccf...,...,Missing,1137f5c95cbd1e9c6513b475f0cc4ebf1df27afa23a6b3...,Suncorp-Metway Limited,VISA,AUS,DEBIT,6e23dbda72e8f0b5236187e648159b5cca78fb829b56f7...,0.503788,0.0,1.000000
4,True,first,0.011686,0.0,0.011279,864d0403fc982633f1c59ec6c7bc135a4332c05b96be46...,4c5c0212b292b788c8d1eadba5fa3e4ebd5b9f8b99ebde...,AUS,en,e2f394fa8f483b12901d45482042087ba8356a469b6b93...,...,Missing,8e003dae05dbbdda3fd51a0b266f4bd8254dbbd539143a...,National Australia Bank Limited,VISA,AUS,DEBIT,63428e9b0a8e0a80b98f02428261e1e24048201f3a74bb...,0.685606,0.0,1.000000


## Encode categorical features

In [18]:
df.head()

,is_subscription,transaction_type,currency_amount,currency_id,amount_scaled,merchant_customer_id,merchant_customer_email,merchant_country,merchant_language,ip_address,...,payment_type,card_id,bank,cardbrand,cardcountry,cardtype,bin,card_exp_relative,month_creating,week_day_creating
0,False,first,0.078350,0.0,0.000921,Missing,db30f9b09fc6db70bd69492bad0993610e034dd0aaf196...,USA,Missing,05ebef83d319bc0e9f8f3e5792ed1e7f9b5e93ac61f356...,...,Missing,55b9df24b0ee3d556bd9d92e3e96cd9cd2020aa889a099...,Intl Hdqtrs-Center Owned,VISA,IND,DEBIT,82178912e9cb7cdfe4397e288172d2ddb5b4ca24de4ad7...,0.625000,0.0,0.000000
1,True,first,0.011686,0.0,0.011279,6b5907bb6b2a4db1e1d8e29ecda26c958148c869ae21e8...,4429eb1d2902632d67042cd8b04855c12499684b25fe40...,AUS,en,efdd674a78604c0efb156dbad042dbe633c08338a1a988...,...,Missing,37ebdd04d53f8bbda6cdc8a67a63a3f07178697cc4b81b...,National Australia Bank Limited,VISA,AUS,CREDIT,c84a4f90930cf35cb2718e49919df4f2400347c3e28e5c...,0.526515,0.0,1.000000
2,True,first,0.011686,0.0,0.011394,6f512fcdad63bdcac2fb2283d54e19cfb37b3714d38786...,af46d0225af51b3083c33d1f31e380e07cfa6d7256f30d...,AUS,en,fdb9dd738153f7893179977c6814f916260ef5e4d8a424...,...,Missing,2a7e39132540646a182b136943d02522d6990e9fdedf09...,National Australia Bank Limited,VISA,AUS,DEBIT,a321caacb48f0dd4ac473a76fb7a8eaa54f0c88d409594...,0.488636,0.0,0.333333
3,True,first,0.062664,0.0,0.060536,2ae2b02e168b5614f46a2b94715562012cf01f405b4da2...,53514d7fdcf50004badc5914c5faaad5311e94b9596c90...,AUS,en,1b487bab850c13007c4b0cecfa3003285e53e676bddccf...,...,Missing,1137f5c95cbd1e9c6513b475f0cc4ebf1df27afa23a6b3...,Suncorp-Metway Limited,VISA,AUS,DEBIT,6e23dbda72e8f0b5236187e648159b5cca78fb829b56f7...,0.503788,0.0,1.000000
4,True,first,0.011686,0.0,0.011279,864d0403fc982633f1c59ec6c7bc135a4332c05b96be46...,4c5c0212b292b788c8d1eadba5fa3e4ebd5b9f8b99ebde...,AUS,en,e2f394fa8f483b12901d45482042087ba8356a469b6b93...,...,Missing,8e003dae05dbbdda3fd51a0b266f4bd8254dbbd539143a...,National Australia Bank Limited,VISA,AUS,DEBIT,63428e9b0a8e0a80b98f02428261e1e24048201f3a74bb...,0.685606,0.0,1.000000


In [19]:
encoders = {}
for col in df.select_dtypes(include=["object"]).columns:
    encoders[col] = LabelEncoder()
    encoders[col].fit(df[col].astype(str))

In [20]:
for col, encoder in encoders.items():
    df[col] = encoder.transform(df[col].astype(str))

In [21]:
df.head()

,is_subscription,transaction_type,currency_amount,currency_id,amount_scaled,merchant_customer_id,merchant_customer_email,merchant_country,merchant_language,ip_address,...,payment_type,card_id,bank,cardbrand,cardcountry,cardtype,bin,card_exp_relative,month_creating,week_day_creating
0,False,0,0.078350,0.0,0.000921,7800194,10674060,123,2,391738,...,1,3901341,6999,8,48,3,24076,0.625000,0.0,0.000000
1,True,0,0.011686,0.0,0.011279,5231861,3320731,6,10,10219550,...,1,2544007,9054,8,5,1,37049,0.526515,0.0,1.000000
2,True,0,0.011686,0.0,0.011394,5425353,8536566,6,10,10556925,...,1,1932961,9054,8,5,3,30165,0.488636,0.0,0.333333
3,True,0,0.062664,0.0,0.060536,2089526,4058781,6,10,1150350,...,1,783692,11789,8,5,3,20438,0.503788,0.0,1.000000
4,True,0,0.011686,0.0,0.011279,6546007,3719303,6,10,9881472,...,1,6463568,9054,8,5,3,18382,0.685606,0.0,1.000000


## Boolean encoder

In [22]:
df[["is_subscription", "is_secured"]] = df[["is_subscription", "is_secured"]].astype(int)

In [23]:
df.head()

,is_subscription,transaction_type,currency_amount,currency_id,amount_scaled,merchant_customer_id,merchant_customer_email,merchant_country,merchant_language,ip_address,...,payment_type,card_id,bank,cardbrand,cardcountry,cardtype,bin,card_exp_relative,month_creating,week_day_creating
0,0,0,0.078350,0.0,0.000921,7800194,10674060,123,2,391738,...,1,3901341,6999,8,48,3,24076,0.625000,0.0,0.000000
1,1,0,0.011686,0.0,0.011279,5231861,3320731,6,10,10219550,...,1,2544007,9054,8,5,1,37049,0.526515,0.0,1.000000
2,1,0,0.011686,0.0,0.011394,5425353,8536566,6,10,10556925,...,1,1932961,9054,8,5,3,30165,0.488636,0.0,0.333333
3,1,0,0.062664,0.0,0.060536,2089526,4058781,6,10,1150350,...,1,783692,11789,8,5,3,20438,0.503788,0.0,1.000000
4,1,0,0.011686,0.0,0.011279,6546007,3719303,6,10,9881472,...,1,6463568,9054,8,5,3,18382,0.685606,0.0,1.000000


## Saving results

In [24]:
df["is_fraud"] = is_fraud
df["transaction_id"] = transaction_id

In [25]:
df.to_csv(output_path)